# Practical Application III: Comparing Classifiers

**Overview**: In this practical application, your goal is to compare the performance of the classifiers we encountered in this section, namely K Nearest Neighbor, Logistic Regression, Decision Trees, and Support Vector Machines.  We will utilize a dataset related to marketing bank products over the telephone.  



### Getting Started

Our dataset comes from the UCI Machine Learning repository [link](https://archive.ics.uci.edu/ml/datasets/bank+marketing).  The data is from a Portugese banking institution and is a collection of the results of multiple marketing campaigns.  We will make use of the article accompanying the dataset [here](CRISP-DM-BANK.pdf) for more information on the data and features.



### Problem 1: Understanding the Data

To gain a better understanding of the data, please read the information provided in the UCI link above, and examine the **Materials and Methods** section of the paper.  How many marketing campaigns does this data represent?

In [36]:
#The paper states the dataset is a result of 17 campaigns.
print("The data represents 17 marketing campaigns.")

The data represents 17 marketing campaigns.


### Problem 2: Read in the Data

Use pandas to read in the dataset `bank-additional-full.csv` and assign to a meaningful variable name.

In [37]:
import pandas as pd

In [38]:
df = pd.read_csv('/home/grace/comparing_classifiers/data/bank-additional-full.csv', sep=';')

In [39]:
df.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y
0,56,housemaid,married,basic.4y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
1,57,services,married,high.school,unknown,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
2,37,services,married,high.school,no,yes,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
3,40,admin.,married,basic.6y,no,no,no,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no
4,56,services,married,high.school,no,no,yes,telephone,may,mon,...,1,999,0,nonexistent,1.1,93.994,-36.4,4.857,5191.0,no


### Problem 3: Understanding the Features


Examine the data description below, and determine if any of the features are missing values or need to be coerced to a different data type.


```
Input variables:
# bank client data:
1 - age (numeric)
2 - job : type of job (categorical: 'admin.','blue-collar','entrepreneur','housemaid','management','retired','self-employed','services','student','technician','unemployed','unknown')
3 - marital : marital status (categorical: 'divorced','married','single','unknown'; note: 'divorced' means divorced or widowed)
4 - education (categorical: 'basic.4y','basic.6y','basic.9y','high.school','illiterate','professional.course','university.degree','unknown')
5 - default: has credit in default? (categorical: 'no','yes','unknown')
6 - housing: has housing loan? (categorical: 'no','yes','unknown')
7 - loan: has personal loan? (categorical: 'no','yes','unknown')
# related with the last contact of the current campaign:
8 - contact: contact communication type (categorical: 'cellular','telephone')
9 - month: last contact month of year (categorical: 'jan', 'feb', 'mar', ..., 'nov', 'dec')
10 - day_of_week: last contact day of the week (categorical: 'mon','tue','wed','thu','fri')
11 - duration: last contact duration, in seconds (numeric). Important note: this attribute highly affects the output target (e.g., if duration=0 then y='no'). Yet, the duration is not known before a call is performed. Also, after the end of the call y is obviously known. Thus, this input should only be included for benchmark purposes and should be discarded if the intention is to have a realistic predictive model.
# other attributes:
12 - campaign: number of contacts performed during this campaign and for this client (numeric, includes last contact)
13 - pdays: number of days that passed by after the client was last contacted from a previous campaign (numeric; 999 means client was not previously contacted)
14 - previous: number of contacts performed before this campaign and for this client (numeric)
15 - poutcome: outcome of the previous marketing campaign (categorical: 'failure','nonexistent','success')
# social and economic context attributes
16 - emp.var.rate: employment variation rate - quarterly indicator (numeric)
17 - cons.price.idx: consumer price index - monthly indicator (numeric)
18 - cons.conf.idx: consumer confidence index - monthly indicator (numeric)
19 - euribor3m: euribor 3 month rate - daily indicator (numeric)
20 - nr.employed: number of employees - quarterly indicator (numeric)

Output variable (desired target):
21 - y - has the client subscribed a term deposit? (binary: 'yes','no')
```



In [53]:
#For this problem, it is clear that some features contain the string “unknown” instead of the usual NaN (Not a Number) values. At this point, 
#I understand how important it is to know my data—machine learning #algorithms cannot process string values like “unknown.” Models such as 
#Logistic Regression and Support Vector Machines rely on mathematical #computations, so non-numeric placeholders can cause errors or distort results.
#
#When I examined the dataset, I found a total of 8,597 “unknown” entries #in the default feature alone. This is a significant number that could 
#lead to issues if left unaddressed. Not handling these “unknown” values #properly could result in feature encoding failures and model bias. 
#For instance, if the model interprets “unknown” in the default column as a #genuine category—similar to “yes” or “no”—it might incorrectly treat 
#missing data as a meaningful client characteristic, skewing the analysis.To prevent this, I decided to treat “unknown” as its own category, which 
#aligns with how the original researchers likely handled these cases.

import pandas as pd

# Load dataset (using consistent variable name)
bank_df = pd.read_csv('/home/grace/comparing_classifiers/data/bank-additional-full.csv', sep=';')

# Check which columns contain the string "unknown"
unknown_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']

# Count "unknown" occurrences per column
unknown_counts = (bank_df[unknown_cols] == 'unknown').sum()

# Add total and percentage info
total_unknowns = unknown_counts.sum()
total_rows = len(bank_df)
percent_unknown = round((total_unknowns / (total_rows * len(unknown_cols))) * 100, 2)
print("Non-standard missing value ('unknown') counts:\n")
print(unknown_counts)
print(f"\nTotal 'unknown' entries across selected columns: {total_unknowns}")
print(f"Average missing percentage (approx.): {percent_unknown}%")

# Check data types for all columns
print("\nData types overview:")
print(bank_df.dtypes)

Non-standard missing value ('unknown') counts:

job           330
marital        80
education    1731
default      8597
housing       990
loan          990
dtype: int64

Total 'unknown' entries across selected columns: 12718
Average missing percentage (approx.): 5.15%

Data types overview:
age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
duration            int64
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp.var.rate      float64
cons.price.idx    float64
cons.conf.idx     float64
euribor3m         float64
nr.employed       float64
y                  object
dtype: object


### Problem 4: Understanding the Task

After examining the description and data, your goal now is to clearly state the *Business Objective* of the task.  State the objective below.

In [54]:
print("The business objective is to predict whether a client will subscribe to a term deposit, \
enabling the bank to improve the efficiency of future marketing campaigns and optimize client targeting.")


The business objective is to predict whether a client will subscribe to a term deposit, enabling the bank to improve the efficiency of future marketing campaigns and optimize client targeting.


### Problem 5: Engineering Features

Now that you understand your business objective, we will build a basic model to get started.  Before we can do this, we must work to encode the data.  Using just the bank information features, prepare the features and target column for modeling with appropriate encoding and transformations.

In [44]:
# In this step, I begin preparing the dataset for modeling by focusing only on the
# bank client information features. These features describe each client’s personal
# and financial profile, which are useful for predicting whether the client will
# subscribe to a term deposit or not.
#
# I import the essential libraries for data preparation and encoding:
#   • pandas and numpy — data manipulation and numerical ops
#   • train_test_split — splits data into train/test
#   • OneHotEncoder and StandardScaler  — transform categorical and numeric variables
#   • ColumnTransformer and Pipeline — combine preprocessing steps cleanly
#
# I define two groups of features:
#   • Numeric: age
#   • Categorical: job, marital, education, default, housing, loan
#
# These seven columns constitute the “bank client information” subset used here.
# I then build a preprocessing pipeline that standardizes numeric features and
# one-hot encodes categoricals (handling unseen categories safely). Finally, I
# split the data with stratification and apply the transformations to produce
# X_train_enc and X_test_enc for downstream modeling.

In [55]:

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
import pandas as pd
import numpy as np

# Load dataset
df = pd.read_csv('/home/grace/comparing_classifiers/data/bank-additional-full.csv', sep=';')

# Define features: only bank client information
num_cols = ['age']
cat_cols = ['job', 'marital', 'education', 'default', 'housing', 'loan']

# Target variable (encode 'yes' -> 1, 'no' -> 0)
y = df['y'].map({'yes': 1, 'no': 0})

# Feature subset
X = df[num_cols + cat_cols].copy()

# Preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ]
)

# Train/test split (preserve class distribution)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42, stratify=y
)

# Fit and transform
X_train_enc = preprocessor.fit_transform(X_train)
X_test_enc  = preprocessor.transform(X_test)

# Optional checks
print("Transformed training data shape:", X_train_enc.shape)
print("Transformed testing data shape:", X_test_enc.shape)
print("Training target distribution:\n", y_train.value_counts(normalize=True))



Transformed training data shape: (28831, 34)
Transformed testing data shape: (12357, 34)
Training target distribution:
 y
0    0.887343
1    0.112657
Name: proportion, dtype: float64


The bank-client subset was successfully preprocessed using one-hot encoding and scaling.
The resulting training data contains 34 features and maintains the original class distribution (≈ 89 % no, 11 % yes).
This encoded dataset is now ready for model development.

### Problem 6: Train/Test Split

With your data prepared, split it into a train and test set.

In [ ]:
# The code below prepares the dataset for modeling by dividing it into training and testing sets 
# and applying preprocessing steps. 
#
# The train_test_split() function separates the data so that 70% is used for training the model 
# and 30% is reserved for testing its performance—this is a common practice to ensure 
# the model can generalize to unseen data. 
#
# The parameter stratify=y ensures that both the training and test sets maintain the same 
# proportion of positive (“yes”) and negative (“no”) outcomes as the original dataset, 
# which is especially important when the classes are imbalanced. 
#
# Next, the preprocessing steps defined earlier are applied: the preprocessor is fit and 
# transformed on the training data to learn the necessary scaling and encoding, and then 
# used to transform the test data so that the same learned transformations are applied consistently.


In [56]:

# The dataset is divided into training (70%) and testing (30%) sets to evaluate model generalization.
# The 'stratify=y' parameter ensures both subsets maintain the same proportion of 
# positive ('yes') and negative ('no') outcomes as the original dataset—important for imbalanced data.

from sklearn.model_selection import train_test_split

# Split features (X) and target (y)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

# Apply preprocessing to training and testing data
X_train_enc = preprocessor.fit_transform(X_train)
X_test_enc  = preprocessor.transform(X_test)

# Confirm split shapes
print("Training set shape:", X_train_enc.shape)
print("Testing set shape:", X_test_enc.shape)
print("Training target distribution:\n", y_train.value_counts(normalize=True))


Training set shape: (28831, 34)
Testing set shape: (12357, 34)
Training target distribution:
 y
0    0.887343
1    0.112657
Name: proportion, dtype: float64


The dataset was successfully divided into training and testing sets, with 28,831 records (approximately 70%) used for training and 12,357 records (around 30%) reserved for testing. This proportion aligns with standard machine learning practices, ensuring that the model has sufficient data to learn from while preserving a representative sample for evaluation. Both the training and testing datasets contain 34 features after preprocessing, confirming that the transformations—such as scaling the numeric feature and applying one-hot encoding to categorical variables—were applied consistently across both subsets.

The target distribution shows that 88.7% of the observations represent clients who did not subscribe to a term deposit, while 11.3% represent those who did. This imbalance reflects the real-world challenge of the dataset, where most clients decline the offer. Stratified sampling successfully maintained this class proportion in both subsets, which is crucial for achieving reliable and unbiased model performance. Overall, the preprocessing and data splitting were completed correctly, resulting in a clean and well-balanced dataset ready for model training and evaluation.

### Problem 7: A Baseline Model

Before we build our first model, we want to establish a baseline.  What is the baseline performance that our classifier should aim to beat?

In [48]:
print("The baseline performance that our classifier should aim to beat is approximately 88-90% accuracy. The model should be able to predict the clients will not subscribe to the a term deposit.")

The baseline performance that our classifier should aim to beat is approximately 88-90% accuracy. The model should be able to predict the clients will not subscribe to the a term deposit.


In this dataset, about 89% of clients did not subscribe to a term deposit. Therefore, a simple model that predicts “no” for every client would achieve around 89% accuracy. This serves as the baseline performance—the minimum level that any predictive model must exceed to be considered useful. However, this naive approach would have poor recall for the minority class (the actual subscribers), making it ineffective for business decisions. The goal of building a machine learning model is therefore to improve predictive power for the positive class (clients who do subscribe) while maintaining or improving overall accuracy.

### Problem 8: A Simple Model

Use Logistic Regression to build a basic model on your data.  

In [57]:
# In this step, I trained a Logistic Regression model to predict whether a bank client 
# will subscribe to a term deposit (y = "yes" or "no") using the encoded personal and 
# financial attributes prepared earlier.
#
# Logistic Regression is a strong baseline for binary classification because it is:
# 
# - Simple and computationally efficient
# - Interpretable through model coefficients
# - Effective with one-hot-encoded categorical features and scaled numerical data
#
# I used class_weight="balanced" to address the class imbalance present in the dataset 
# (only about 11% of clients subscribed — a common issue in the original study).
#
# After training on the processed features (X_train_enc, y_train), the model was evaluated 
# on the test set. The accuracy score and the classification report summarize how well it 
# predicts compared to the baseline accuracy from Problem 7 (≈ 88–90%).
#
# If the Logistic Regression model identifies more positive (“yes”) cases while maintaining 
# or improving accuracy, it provides meaningful predictive value beyond random or 
# majority-class guessing.


from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Initialize Logistic Regression model
log_reg = LogisticRegression(max_iter=1000, random_state=42, class_weight='balanced')

# Fit the model
log_reg.fit(X_train_enc, y_train)

# Predict on the test data
y_pred = log_reg.predict(X_test_enc)

# Evaluate performance
log_reg_accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy:", log_reg_accuracy)

print("\nClassification Report:\n", classification_report(y_test, y_pred, zero_division=0))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Logistic Regression Accuracy: 0.593590677348871

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.59      0.72     10965
           1       0.16      0.62      0.26      1392

    accuracy                           0.59     12357
   macro avg       0.54      0.61      0.49     12357
weighted avg       0.84      0.59      0.67     12357


Confusion Matrix:
 [[6466 4499]
 [ 523  869]]


The Logistic Regression model only achieved 59.4% as expected since I used class_weight='balanced' I had to assign more weight to the minority class ("yes") so the model will shift focus from simply maximizing accuracy to better identify potential subscribers.

### Problem 9: Score the Model

What is the accuracy of your model?

In [58]:
print("Model Accuracy: 59.36%")


Model Accuracy: 59.36%


The Logistic Regression model achieved an accuracy of approximately 59.36% on the test set.
While this is lower than the 89% baseline, this result reflects a balanced trade-off between precision and recall after addressing the class imbalance with class_weight='balanced'.
In imbalanced datasets like this, accuracy alone is not a reliable indicator of model performance — the recall for the minority class (“yes”) is a more meaningful measure of success at this stage. The model correctly identified six out of ten clients overall, while significantly improving reliability to recognize potential subscribers.

### Problem 10: Model Comparisons

Now, we aim to compare the performance of the Logistic Regression model to our KNN algorithm, Decision Tree, and SVM models.  Using the default settings for each of the models, fit and score each.  Also, be sure to compare the fit time of each of the models.  Present your findings in a `DataFrame` similar to that below:

| Model | Train Time | Train Accuracy | Test Accuracy |
| ----- | ---------- | -------------  | -----------   |
|    |    |.     |.     |

In [59]:
# Problem 10 — Model Comparisons

import time
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from scipy import sparse


def maybe_dense(X, model_name):
    if model_name in {"KNN", "SVM"} and sparse.issparse(X):
        return X.toarray()
    return X

models = {
    "Logistic Regression": LogisticRegression(max_iter=1000, random_state=42),  
    "KNN": KNeighborsClassifier(),                                             
    "Decision Tree": DecisionTreeClassifier(random_state=42),                  
    "SVM": SVC(random_state=42)                                                
}

rows = []
for name, model in models.items():
    Xtr = maybe_dense(X_train_enc, name)
    Xte = maybe_dense(X_test_enc,  name)

    t0 = time.time()
    model.fit(Xtr, y_train)
    train_time = time.time() - t0

    train_acc = model.score(Xtr, y_train)
    test_acc  = model.score(Xte, y_test)

    rows.append({
        "Model": name,
        "Train Time (s)": round(train_time, 4),
        "Train Accuracy": round(train_acc, 4),
        "Test Accuracy": round(test_acc, 4)
    })

results_df = pd.DataFrame(rows).sort_values(by="Test Accuracy", ascending=False).reset_index(drop=True)
results_df


,Model,Train Time (s),Train Accuracy,Test Accuracy
0,Logistic Regression,0.0971,0.8873,0.8874
1,SVM,45.1051,0.8882,0.8868
2,KNN,0.0032,0.8915,0.8777
3,Decision Tree,0.4172,0.9188,0.8642


Four models were trained and compared using default parameters to establish baseline performance and efficiency. Logistic Regression achieved a test accuracy of 88.74%, which is almost identical to the Support Vector Machine (SVM) model at 88.68%. While SVM produced slightly higher training accuracy, its training time was significantly longer (about 45 seconds versus less than a second for Logistic Regression). This reflects SVM’s higher computational cost due to the complexity of finding the optimal separating boundary, especially in high-dimensional data.

The K-Nearest Neighbors (KNN) model achieved comparable accuracy (87.77%) but required minimal training time since it performs most of its computation during prediction rather than fitting. Meanwhile, the Decision Tree model recorded the highest training accuracy (91.88%) but the lowest test accuracy (86.42%), indicating signs of overfitting — the model fits the training data too closely and generalizes poorly to unseen data.

Overall, all models performed similarly in terms of test accuracy, clustering around the baseline level (~88%), but SVM demonstrated strong generalization and stability across both training and testing sets. This performance pattern supports the original researchers’ preference for SVM, as it provided the most consistent and reliable predictive power, albeit with a longer training time.

### Problem 11: Improving the Model

Now that we have some basic models on the board, we want to try to improve these.  Below, we list a few things to explore in this pursuit.


- Hyperparameter tuning and grid search.  All of our models have additional hyperparameters to tune and explore.  For example the number of neighbors in KNN or the maximum depth of a Decision Tree.  
- Adjust your performance metric

##### Questions